# Analyse planetary nebual
- author : Sylvie Dagoret-Campagne
- affiliation : IJCLab/IN2P3/CNES, DESC-LSST fullmember, behalf AuxTel Teal VERA C. Rubin observatory
- creation date : 16 September2021
- **kernel : anaconda3_auxtel**
- update : July 22th 2021
- update : September 16th 2021 



# Selection Flag

In [ ]:
from astropy.io import fits
import sys, platform, os
#from astropy.visualization import simple_norm
#from astropy.visualization import imshow_norm, MinMaxInterval, SqrtStretch
import astropy.visualization  as aspv
from astropy.wcs import WCS
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import re
from matplotlib.colors import LogNorm

In [ ]:
from astroquery.simbad import Simbad
from astropy.coordinates import SkyCoord

In [ ]:
plt.rcParams["axes.labelsize"]="large"
plt.rcParams["axes.linewidth"]=2.0
plt.rcParams["xtick.major.size"]=8
plt.rcParams["ytick.major.size"]=8
plt.rcParams["ytick.minor.size"]=5
plt.rcParams["xtick.labelsize"]="large"
plt.rcParams["ytick.labelsize"]="large"

plt.rcParams["figure.figsize"]=(12,12)
plt.rcParams['axes.titlesize'] = 16
plt.rcParams['axes.titleweight'] = 'bold'
#plt.rcParams['axes.facecolor'] = 'blue'
plt.rcParams['xtick.direction'] = 'out'
plt.rcParams['ytick.direction'] = 'out'
plt.rcParams['lines.markeredgewidth'] = 0.3 # the line width around the marker symbol
plt.rcParams['lines.markersize'] = 5  # markersize, in points
plt.rcParams['grid.alpha'] = 0.75 # transparency, between 0.0 and 1.0
plt.rcParams['grid.linestyle'] = '-' # simple line
plt.rcParams['grid.linewidth'] = 0.4 # in points
plt.rcParams['font.size'] = 13

# Target

In [ ]:
target_name="PNG321.0+03.9"

In [ ]:
target_coord=SkyCoord.from_name(target_name)
target_coord

In [ ]:
tagname=".*(PNG).*"

# Work

In [ ]:
#intputdir_day_1="/sps/lsst/groups/auxtel/data/2021/holo/localreduced/2021-07-07/"   
intputdir="/sps/lsst/groups/auxtel/data/2021/holo/quickLookExp/2021-09-09/"   
list_of_files=os.listdir(intputdir)    

In [ ]:
print(list_of_files)

In [ ]:
bad_files = []
idx=0
for file in list_of_files:
    if re.search(".*(PNG).*",file):
        print(file)
    else:
        bad_files.append(idx)
        
    idx+=1
        

In [ ]:
#list_of_file=np.array(list_of_files)
list_of_files=np.delete(list_of_files,bad_files)

In [ ]:
idx_file=0

In [ ]:
fits_image_filename=list_of_files[idx_file]
full_fits_image_filename=os.path.join(intputdir,fits_image_filename)

In [ ]:
fits_image_filename

In [ ]:
hdul = fits.open(full_fits_image_filename)

In [ ]:
hdul.info()

In [ ]:
image=hdul[0].data
header=hdul[0].header

In [ ]:
header

In [ ]:
image.shape

In [ ]:
fig=plt.figure(figsize=(12,12))
ax=fig.add_subplot(1,1,1)

ax.imshow(image,origin="lower",norm=LogNorm(),cmap="gray")

ax.minorticks_on()
ax.grid(color="k")
ax.set_title(fits_image_filename)

# Spectrum

In [ ]:
target_name="PNG321.0+03.9"

In [ ]:
target_coord=SkyCoord.from_name(target_name)
target_coord